## 노드 정의

In [2]:
from langchain_chroma import Chroma
from langgraph.prebuilt import ToolNode
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import tool
from crawler import google_crawler

def get_tool_by_collection(collection_name: str):
    vectorstore = Chroma(collection_name)
    retriever = vectorstore.as_retriever()
    return create_retriever_tool(
        retriever,
        f"retrieve_{collection_name}"
        f"retriever tool for {collection_name} chromaDB collection",
    )

@tool
def google_search_tool(query: str):
    return google_crawler.search(query=query)


tools = [get_tool_by_collection("cnu_post_data"),
         get_tool_by_collection("cnu_post_content_data"),
         google_search_tool]

tool_node = ToolNode(tools)

In [ ]:
from llm_models import Llama38BKorModel
from state import State, graph_builder
from langgraph.graph import START, END

llm = Llama38BKorModel()


# Define the function that determines whether to continue or not
def should_continue(state: State) -> Literal["tools", END]:
    
    # TODO: 스코어가 낮을 때 다시 돌아갈 것
    

    if last_message.tool_calls:
        return "retrieve"
    
    return END

def answer(state: State):
    return {"answer": llm.query(state["question"])}

def question(state: State):
    query =  "수강신청 기간 언제야?"
    return {"question": query}

def retrieve(state: State):

    

    return {"context": [llm.query(state["question"])]}

def binary_score(state: State):
    return {"binary_score": [llm.query(state["question"])]}



# 노드의 추가
graph_builder.add_node("answer", answer)

# chatbot 노드의 이전 경로로 START를 포함함
graph_builder.add_edge(START, "answer")

# chatbot 노드의 다음 경로로 END를 포함함
graph_builder.add_edge("answer", END)


graph = graph_builder.compile()

In [8]:
from langgraph.graph import END, StateGraph, MessagesState

# Define the function that determines whether to continue or not
def should_continue(state: GraphState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}



NameError: name 'MessagesState' is not defined